In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys
import time
import scipy.linalg as spla
from lib.compute_KCL_matrices import compute_KCL_matrices
from lib.basematrices import basematrices
from lib.NR3_timevarying import NR3_timevarying

In [ ]:
fn = '06node_threephase_unbalancedVR.dss'
dss.run_command('Redirect ' + fn)
print(dss.Lines.AllNames())
print(dss.Circuit.AllBusNames())

In [ ]:
dss.Lines.AllNames()
dss.Circuit.AllBusNames()

def linelist(busname):
        in_lines = np.array([])
        out_lines = np.array([])
        for k in range(len(dss.Lines.AllNames())):
            dss.Lines.Name(dss.Lines.AllNames()[k])
            if busname in dss.Lines.Bus1():
                out_lines = np.append(out_lines, dss.Lines.AllNames()[k])
            elif busname in dss.Lines.Bus2():
                in_lines = np.append(in_lines, dss.Lines.AllNames()[k])
        return in_lines,out_lines
linelist(dss.Circuit.AllBusNames()[2])

Vectorized Solver

In [2]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = '06node_threephase_radial_unbalanced.dss'
#fn = 'IEEE_13Node_Modified_01.dss'
fn = '06node_threephase_meshed_unbalanced.dss'
fn = '06node_threephase_unbalancedVR.dss'
fn = '06node_threephase_radial_unbalanced_VR.dss'
#fn = '02node_threephase_unbalanced_new.dss'


# Time-steps
times = np.linspace(0, 2*np.pi, 5)
# DER changes over time-steps
der = [0, 0, 0, 0, 0]
# Capacitance changes over time-steps
capacitance = [0, 0, 0, 0, 0]

# Pre-generate matrices
t00 = time.time()
X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, H_reg, G_reg = basematrices(fn, slackidx, Vslack, None, None)

t0 = time.time()
for _ in range(1):   
    dss.run_command('Redirect ' + fn)
    nnode = len(dss.Circuit.AllBusNames())
    print(nnode)
    nline = len(dss.Lines.AllNames())

    VNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
    INR01=  np.zeros((len(times), 3, nline), dtype = "complex")
    STXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
    SRXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
    iNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
    sNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")

    for i in range(len(times)):
        # Run NR3 with variations in time, DER, and capacitance
        VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
            NR3_timevarying(fn, X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, None, None, der[i], capacitance[i], times[i], H_reg, G_reg)    
        VNR01[i, :, :] = np.reshape(VNR, (3, nnode))
        INR01[i, :, :] = np.reshape(INR, (3, nline))
        STXNR01[i, :, :] = np.reshape(STXNR, (3, nline))
        SRXNR01[i, :, :] = np.reshape(SRXNR, (3, nline))
        iNR01[i, :, :] = np.reshape(iNR, (3, nnode))
        sNR01[i, :, :] = np.reshape(sNR, (3, nnode))
t1 = time.time()

7
Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
Iteration number 4.000000
Iteration number 5.000000
Iteration number 6.000000
Iteration number 7.000000
VNR
[[ 1.        +0.00000000e+00j  0.99741697-9.23444462e-04j
   0.94754612-8.77272238e-04j  0.93955738+8.03578444e-03j
   0.94204554+8.03328788e-03j  0.93074306+1.32109169e-02j
   0.        +0.00000000e+00j]
 [-0.5       -8.66025404e-01j -0.50103146-8.70001487e-01j
  -0.50103146-8.70001487e-01j -0.4977151 -8.80523592e-01j
   0.        +0.00000000e+00j -0.50077875-8.85098838e-01j
  -0.50051704-8.83921259e-01j]
 [-0.5       +8.66025404e-01j -0.50303986+8.60533868e-01j
  -0.55334384+9.46587255e-01j -0.56539047+9.34770905e-01j
  -0.56568729+9.27997161e-01j  0.        +0.00000000e+00j
   0.        +0.00000000e+00j]]
INR:
[[ 4.68252232e-01-0.10525662j  4.80244059e-01-0.10794623j
   4.68252232e-01-0.10525662j  3.74732816e-01-0.0814646j
   1.06370548e-01-0.02563092j  1.61891447e-01-0.05

[[ 4.86517973e-01+0.10998442j  4.97740751e-01+0.11201205j
   4.85312669e-01+0.10922151j  3.69007624e-01+0.08044993j
   1.03560496e-01+0.02589266j  1.57594612e-01+0.05150528j
  -6.94660626e-31+0.j        ]
 [ 1.20804880e-01-0.13752764j  3.61014779e-01+0.07119578j
   1.21458362e-01-0.13795852j  2.57168689e-01+0.04523425j
   1.26057526e-25+0.j          1.54933944e-01+0.04178902j
   5.19788276e-02+0.01599871j]
 [ 1.30471980e-01+0.02774957j  2.84952415e-01+0.04955629j
   1.29879641e-01+0.02838659j  2.09601566e-01+0.0285504j
   1.04498703e-01+0.02572576j  8.71285995e-26+0.j
   0.00000000e+00+0.j        ]]
SRXNR:
[[0.48531267+0.10922151j 0.47285371+0.10641145j 0.46104704+0.10376043j
  0.3650012 +0.08335946j 0.10384609+0.02596152j 0.15576913+0.05192304j
  0.        +0.j        ]
 [0.12145836-0.13795852j 0.36101478+0.07119578j 0.12145836-0.13795852j
  0.25878003+0.04775054j 0.        +0.j         0.15582492+0.04196023j
  0.05192304+0.01596152j]
 [0.12987964+0.02838659j 0.31344766+0.05451192j 0.

In [ ]:
print('allocate base matrices ', t0 - t00)
print('run newton rapshon 20x ', t1-t0)
print(t1-t00)

OpenDSS Solution

In [ ]:
t0_dss = time.time()
for _ in range(1):
    dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
    #dss.run_command('Redirect 06node_threephase_unbalanced.dss') 
    #dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
    dss.run_command('Redirect 06node_threephase_unbalancedVR.dss')
    #dss.run_command('Redirect 02node_threephase_unbalanced_new.dss')

    VDSS0 = np.zeros((len(times), 3, nnode), dtype = "complex")
    IDSS0 =  np.zeros((len(times), 3, nline), dtype="complex")
    SRXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
    STXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
    load_arr = np.zeros((len(times), 3, nnode), dtype='complex')
    kW_list = np.array([])
    kvar_list = np.array([])

    # Store the loads
    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])
        kW_list = np.append(kW_list, dss.Loads.kW())
        kvar_list = np.append(kvar_list, dss.Loads.kvar())

    for i in range(len(times)):
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
        Vbase = dss.Bus.kVBase() * 1000
        Sbase = 1000000.0
        Ibase = Sbase/Vbase
        Zbase = Vbase/Ibase

        # Set slack bus (sourcebus) voltage reference in p.u.
        SlackBusVoltage = 1.000
        dss.Vsources.PU(SlackBusVoltage)

        # Time-varying load
        for k in range(len(dss.Loads.AllNames())):
            dss.Loads.Name(dss.Loads.AllNames()[k])
            dss.Loads.kW(kW_list[k]* (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
            dss.Loads.kvar(kvar_list[k] * (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))

        dss.Solution.Convergence(0.000000000001)

        # Solve power flow with OpenDSS file
        dss.Solution.Solve()
        if not dss.Solution.Converged:
            print('Initial Solution Not Converged. Check Model for Convergence')
        else:
            print('Initial Model Converged. Proceeding to Next Step.')
            #Doing this solve command is required for GridPV, that is why the monitors
            #go under a reset process
            dss.Monitors.ResetAll()

            # set solution Params
            #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
            dss.Solution.Mode(1)
            dss.Solution.Number(1)
            dss.Solution.StepSize(1)
            dss.Solution.ControlMode(-1)
            dss.Solution.MaxControlIterations(1000000)
            dss.Solution.MaxIterations(30000)
            # Easy process to get all names and count of loads, a trick to avoid
            # some more lines of code
            TotalLoads=dss.Loads.Count()
            AllLoadNames=dss.Loads.AllNames()
            print('OpenDSS Model Compliation Done.')
            print('Iterations: ', dss.Solution.Iterations())
            print('Tolerance: ', dss.Solution.Convergence())

        print('')

        # OpenDSS loads
        for k in range(len(dss.Loads.AllNames())):
            dss.Circuit.SetActiveElement('Load.'+dss.Loads.AllNames()[k])
            pattern =  r"(\w+)\."
            load_bus = re.findall(pattern, dss.CktElement.BusNames()[0])
            idxbs = dss.Circuit.AllBusNames().index(load_bus[0]) 
            load_ph_arr_temp = [0, 0, 0] 
            for ph1 in range(1, 4): 
                pattern = r"\.%s" % (str(ph1))
                load_ph = re.findall(pattern, dss.CktElement.BusNames()[0])
                if load_ph:
                    load_ph_arr_temp[ph1 - 1] = 1
            n= 0
            for m in range(len(load_ph_arr_temp)):
                if load_ph_arr_temp[m] == 1:
                    load_arr[i, m, idxbs] +=  (dss.CktElement.Powers()[n] + 1j * dss.CktElement.Powers()[n + 1])*1e3/1e6     #where 1e6 is sbase 
                    n += 2

        print(dss.Solution.Converged())

        # Print number of buses, and bus names
        print(len(dss.Circuit.AllBusNames()))
        print(dss.Circuit.AllBusNames())

        # Print number of loads, and load names
        print(len(dss.Loads.AllNames()))
        print(dss.Loads.AllNames())

        print('')

        VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

        for k1 in range(len(dss.Circuit.AllBusNames())):

            dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])

        #     print(dss.Circuit.AllBusNames()[k1])
        #     print(dss.Bus.Nodes())

        #     print('puVOTLAGES - LN CARTESIAN')
        #     print(dss.Bus.PuVoltage())

            ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1

            Vtemp = np.asarray(dss.Bus.PuVoltage())

            Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]


        #     print(np.asarray(dss.Bus.Nodes(),'int'))

            VDSS[ph,k1] = Vtemp    

        #     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


        #     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
        #     for k2 in range(len(dss.Bus.Nodes())):
        #         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

        print('VDSS\n', np.round(VDSS,decimals=6))


        IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

        for k1 in range(len(dss.Lines.AllNames())):
            dss.Lines.Name(dss.Lines.AllNames()[k1])
        #     print(dss.Lines.AllNames()[k1])
            ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
            Imn = np.asarray(dss.CktElement.Currents())/Ibase
        #     print(Imn)
            Imn = Imn[0:int(len(Imn)/2)]
        #     print(Imn)
            Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
        #     print(Imn)
            IDSS[ph,k1] = Imn
        #     print('')

        print('IDSS\n', np.round(IDSS,decimals=6))

        STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
        SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

        for k1 in range(len(dss.Lines.AllNames())):
            dss.Lines.Name(dss.Lines.AllNames()[k1])
            ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
            Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)

            STXtemp = Sk[0:int(len(Sk)/2)]
            SRXtemp = Sk[int(len(Sk)/2):]

            STXtemp = STXtemp[0:len(ph)*2-1:2] + 1j*STXtemp[1:len(ph)*2:2]
            SRXtemp = -(SRXtemp[0:len(ph)*2-1:2] + 1j*SRXtemp[1:len(ph)*2:2])

            STXDSS[ph,k1] = STXtemp
            SRXDSS[ph,k1] = SRXtemp

        VDSS0[i, :, :] = VDSS
        IDSS0[i, :, :] = IDSS
        SRXDSS0[i, :, :] = SRXDSS
        STXDSS0[i, :, :] = STXDSS


        print('STXDSS\n', np.round(STXDSS,decimals=6))
        print('SRXDSS\n', np.round(SRXDSS,decimals=6))

        print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
        print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
        print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))
t1_dss = time.time()

In [3]:
Vbase = dss.Bus.kVBase() * 1000
Sbase = 1000000.0
Ibase = Sbase/Vbase
Zbase = Vbase/Ibase

In [22]:
# Set global variables and parameters, and test OpenDSS with network
VDSS0 = np.zeros((len(times), 3, nnode), dtype = "complex")
IDSS0 =  np.zeros((len(times), 3, nline), dtype="complex")
SRXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
STXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
load_arr = np.zeros((len(times), 3, nnode), dtype='complex')

#######################################################
###Global variable initialization and error checking###
#######################################################

# Sbase=1
# LoadScalingFactor = 3
# GenerationScalingFactor = 5

# NoiseMultiplyer= 1
# #Set simulation analysis period - the simulation is from StartTime to EndTime
# StartTime = 40600
# EndTime = StartTime + 640
# EndTime += 1 # creating a list, last element does not count, so we increase EndTime by 1
# #Set hack parameters
# TimeStepOfHack = 160
# PercentHacked = np.array([0,0,0,0,0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0])

# #Set initial VBP parameters for uncompromised inverters
# VBP_normal=np.array([0.97, 1, 1, 1.03, 1.06])
# VBP_attack=np.array([0.997, 1, 1, 1.003, 1.007])

# #Set delays for each node
# Delay_VBPCurveShift = (30+2*np.random.randn(31)).astype(int)
# #Delay_VBPCurveShift = (10+2*np.random.randn(31)).astype(int)
# lpf_meas_vector = (1+0.2*np.random.randn(31))
# lpf_output_vector= (0.1+0.015*np.random.randn(31))

# #Set observer voltage threshold
# ThreshHold_vqvp = 0.06
# adaptive_gain=800


# power_factor=0.9
# pf_converted=tan(acos(power_factor))
# # Number_of_Inverters = 13 #even feeder is 34Bus, we only have 13 inverters




# #Error checking of the global variable -- TODO: add error handling here!
# if EndTime < StartTime or EndTime < 0 or StartTime < 0:
#     print('Setup Simulation Times Inappropriately.')
# if NoiseMultiplyer < 0:
#     print('Setup Noise Multiplyer Correctly.')

# Select OpenDSS file
# dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
# dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
# dss.run_command('Redirect 02node_threephase_unbalanced.dss')
dss.run_command('Redirect 06node_threephase_radial_unbalanced_regulator.dss')
# dss.run_command('Redirect 06node_threephase_meshed_unbalanced.dss')
# dss.run_command('Redirect IEEE13Nodeckt.dss')

# Set slack bus (sourcebus) voltage reference in p.u.
SlackBusVoltage = 1.00
dss.Vsources.PU(SlackBusVoltage)

# Solve power flow with OpenDSS file
dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('')

print(dss.Settings.VoltageBases())
print(dss.Vsources.BasekV())

print('')

# print(dss.Solution.Converged())

# Print number of buses, and bus names
print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

# Print number of loads, and load names
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    print(dss.RegControls.TapNumber())
    dss.RegControls.TapNumber(8)
    print(dss.RegControls.TapNumber())
    print('')

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')

print('')

# Print different voltage measurements
for k1 in range(len(dss.Circuit.AllBusNames())):
    if dss.Circuit.AllBusNames()[k1] == 'sourcebus':
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        source_voltage = dss.Bus.PuVoltage()
        print(dss.Circuit.AllBusNames()[k1])
        print('BASE - LN')
        print(dss.Bus.kVBase())
        print('VMAGANGLE - LN POLAR')
        print(dss.Bus.VMagAngle())
        print('puVMAGANGLE - LN POLAR')
        print(dss.Bus.puVmagAngle())
        print('VOTLAGES - LN CARTESIAN')
        print(dss.Bus.Voltages())
        print('puVOTLAGES - LN CARTESIAN')
        print(dss.Bus.PuVoltage())
        print('VLL - LL CARTESIAN')
        print(dss.Bus.VLL())
        print('puVLL - LL CARTESIAN')
        print(dss.Bus.puVLL())
        
        tempvoltage = dss.Bus.Voltages()
        
        Vab = tempvoltage[0]
        
print('')
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print(dss.Bus.Nodes())
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    print('')
    
for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
    print(dss.Lines.AllNames()[k1])
    print(dss.Lines.Bus1(), ' - ', dss.Lines.Bus2())
    print(dss.Lines.Length())
    Sk = dss.CktElement.Powers()
    print(Sk)
    # print(dss.CktElement.YPrim())
    print(dss.Lines.Phases())
    YPrim = np.asarray(dss.CktElement.YPrim())
    YPrim = YPrim[0::2] + 1j*YPrim[1::2]
    YPrim = YPrim.reshape(2*dss.Lines.Phases(),2*dss.Lines.Phases())
    print(YPrim)
    YPrim = YPrim[0:dss.Lines.Phases(),0:dss.Lines.Phases()]
    print(YPrim)

    print('')

for k1 in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[k1])
    print(dss.Loads.AllNames()[k1])
    print(dss.CktElement.BusNames())
    print(dss.CktElement.NodeOrder())
    print(dss.CktElement.Powers())
    print('')

for k1 in range(len(dss.Capacitors.AllNames())):
    print(dss.Capacitors.AllNames()[k1])
    dss.Capacitors.Name(dss.Capacitors.AllNames()[k1])
    print(dss.CktElement.BusNames())
    print(dss.CktElement.NodeOrder())
    print(dss.CktElement.Powers())
    print('')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    print(dss.RegControls.TapNumber())
    print('')

bus1 = 'a01'
bus2 = 'a02'

dss.Circuit.SetActiveBus(bus1)
V1 = np.asarray(dss.Bus.PuVoltage())
print(V1)

dss.Circuit.SetActiveBus(bus2)
V2 = np.asarray(dss.Bus.PuVoltage())
print(V2)

print(V2/V1)
    
# nnode = len(dss.Circuit.AllBusNames())
# nline = len(dss.Lines.AllNames())

# print(source_voltage)
# print(nline)



VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

for k1 in range(len(dss.Circuit.AllBusNames())):

    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])

#     print(dss.Circuit.AllBusNames()[k1])
#     print(dss.Bus.Nodes())

#     print('puVOTLAGES - LN CARTESIAN')
#     print(dss.Bus.PuVoltage())

    ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1

    Vtemp = np.asarray(dss.Bus.PuVoltage())

    Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]


#     print(np.asarray(dss.Bus.Nodes(),'int'))

    VDSS[ph,k1] = Vtemp    

#     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


#     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
#     for k2 in range(len(dss.Bus.Nodes())):
#         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

print('VDSS\n', np.round(VDSS,decimals=6))


IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
#     print(dss.Lines.AllNames()[k1])
    ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    Imn = np.asarray(dss.CktElement.Currents())/Ibase
#     print(Imn)
    Imn = Imn[0:int(len(Imn)/2)]
#     print(Imn)
    Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
#     print(Imn)
    IDSS[ph,k1] = Imn
#     print('')

print('IDSS\n', np.round(IDSS,decimals=6))

STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
    ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)

    STXtemp = Sk[0:int(len(Sk)/2)]
    SRXtemp = Sk[int(len(Sk)/2):]

    STXtemp = STXtemp[0:len(ph)*2-1:2] + 1j*STXtemp[1:len(ph)*2:2]
    SRXtemp = -(SRXtemp[0:len(ph)*2-1:2] + 1j*SRXtemp[1:len(ph)*2:2])

    STXDSS[ph,k1] = STXtemp
    SRXDSS[ph,k1] = SRXtemp

VDSS0[0, :, :] = VDSS
# IDSS0[i, :, :] = IDSS
# SRXDSS0[i, :, :] = SRXDSS
# STXDSS0[i, :, :] = STXDSS
curr = np.array([])
for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    print(dss.RegControls.TapNumber())
    print(dss.RegControls.Transformer())
    dss.Transformers.Name(dss.RegControls.Transformer())
    print(np.divide(dss.CktElement.Currents(),Ibase))
    curr = np.append(curr, [dss.CktElement.Currents()])
    print('')

    

IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
#     print(dss.Lines.AllNames()[k1])
    #ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    Imn = np.asarray(dss.CktElement.Currents())/Ibase
#     print(Imn)
    Imn = Imn[0:int(len(Imn)/2)]
#     print(Imn)
    Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
#     print(Imn)
    IDSS[ph,k1] = Imn

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.

[4.8]
4.8

7
['sourcebus', 'a01', 'a02', 'a03', 'a04', 'a05', 'a06']
9
['load_a02_a_01', 'load_a02_b_01', 'load_a02_c_01', 'load_a03_abc_01', 'load_a04_a_01', 'load_a04_c_01', 'load_a05_ab_01', 'load_a05_a_01', 'load_a06_b_01']
reg1
0
8

reg2
0
8

reg3
0
8


sourcebus
BASE - LN
2.7712812921102037
VMAGANGLE - LN POLAR
[2771.272850614909, -0.0003529041732815008, 2771.2748240843853, -120.00028103538104, 2771.2764459839786, 119.999752952218]
puVMAGANGLE - LN POLAR
[0.9999969539377621, -0.0003529041732815008, 0.9999976660522204, -120.00028103538104, 0.9999982513048247, 119.999752952218]
VOTLAGES - LN CARTESIAN
[2771.272850562342, -0.017069211077001697, -1385.649184250476, -2399.9876017992115, -1385.6278749538255, 2400.0017775444535]
puVOTLAGES - LN CARTESIAN
[0.9999969539187935, -6.1593210063509205e-06, -0.5000030809558735, -0.866020929969087, -0.49999539162577517, 0.8660260452005442]
VLL - LL CARTESIAN
[4156

In [35]:
curr_Reshaped = curr.reshape((3, len(curr)//3))
print(curr_Reshaped)
curr_reshaped_selected = curr_Reshaped[:, 0] + 1j * curr_Reshaped[:, 1]
print("\n")
print(curr_reshaped_selected)
Ibasecurr = curr_reshaped_selected / Ibase

[[ 164.18176914  -35.74245425 -164.18176422   35.74065266 -156.36357981
    34.03682516  156.36358498  -34.03871682]
 [ -83.78645402  -98.59193701   83.78488759   98.59283607   79.79348631
    93.89888313  -79.79513105  -93.89793912]
 [ -39.19740574  103.22897094   39.19896343 -103.22806343   37.33398169
   -98.31148848  -37.33234612   98.31244136]]


[164.18176914 -35.74245425j -83.78645402 -98.59193701j
 -39.19740574+103.22897094j]


In [34]:
print(INR01[0][:, 1])
#print(INR01[0][:, 2])
print(Ibasecurr)
print(np.max(np.abs(Ibasecurr - INR01[0][:,1])))

[ 0.43331158-0.0977546j  -0.22071317-0.25439169j -0.10240686+0.27391235j]
[ 0.45499387-0.09905239j -0.23219583-0.27322599j -0.10862704+0.28607652j]
0.022058615820160807


In [17]:
INR01[0]

array([[ 4.44132061e-01-0.10019106j,  4.33311583e-01-0.0977546j ,
         4.44132061e-01-0.10019106j,  3.37858693e-01-0.07380887j,
         9.59619685e-02-0.02319167j,  1.45813469e-01-0.04656j   ,
        -2.67509087e-28+0.j        ],
       [ 4.88869637e-02-0.1655659j , -2.20713165e-01-0.25439169j,
         4.88869637e-02-0.1655659j , -1.52800352e-01-0.18400443j,
        -3.20262621e-27+0.j        , -1.02214183e-01-0.10416556j,
        -3.54659782e-02-0.0339661j ],
       [-3.91056131e-02+0.12281761j, -1.02406864e-01+0.27391235j,
        -3.91056131e-02+0.12281761j, -7.48398782e-02+0.17938616j,
        -2.94383957e-02+0.09484255j, -4.33227388e-27+0.j        ,
         0.00000000e+00+0.j        ]])

In [8]:
VDSS0

array([[[ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j],
        [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j],
        [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j]],

       [[ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j],
        [ 0.        +0.00000000e+00j, 

In [5]:
np.max(np.abs(VNR01[0] - VDSS))

0.0038942736975651447

In [6]:
times

array([0.        , 1.57079633, 3.14159265, 4.71238898, 6.28318531])

In [ ]:
VDSS0

Comparing results between three solutions - DSS, 01 (vectorized), 02 (nonvectorized)

In [10]:
for i in range(len(times)):
    print("Timestep %d \n" % i)
    print('Complex Voltage difference')
    #print(np.max(np.abs(VNR01[i] - VNR02[i])))
    print(np.max(np.abs(VDSS0[i] - VNR01[i]))) 

#     print('Complex Current difference')
#     print(np.max(np.abs(IDSS0[i] - INR01[i])))
    
#     print('Complex TX Power difference')
#     print(np.max(np.abs(STXDSS0[i] - STXNR01[i])))
    
#     print('Complex RX Power difference')
#     print(np.max(np.abs(SRXDSS0[i] - SRXNR01[i])))
    
#     print('Voltage Magnitude difference')
#     print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR01[i]))))
    
#     print('Voltage Angle difference')
#     print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR01[i]))))
#     print("\n")

Timestep 0 

Complex Voltage difference
0.0038942736975651447
Timestep 1 

Complex Voltage difference
1.0642607865677964
Timestep 2 

Complex Voltage difference
1.0644091340443609
Timestep 3 

Complex Voltage difference
1.0645546744714
Timestep 4 

Complex Voltage difference
1.0646959843755264


In [ ]:
np.abs(VDSS0[0]-VNR01[0])
print(np.abs(VNR01[0]))

Load comparison between non-vectorized and OpenDSS

In [ ]:
for i in range(len(times)):
    print("Timestep %d \n Magnitude of the Difference: \n" % i)
    print(np.abs(spah[i] - load_arr[i]))
    print("Max difference: \n ")
    print(np.max(np.abs(spah[i] - load_arr[i])))
    print("\n")

In [ ]:
dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
for i in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[i])
    print(dss.Loads.kW())
    print(dss.Loads.kvar())
    print(dss.CktElement.Powers())

---

In [ ]:
t = -1
def load_order_f():
    load_order = {}
    for n in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[n])
        pattern =  r"(\w+)\."
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0])
        if load_bus[0] not in load_order:
            load_order[load_bus[0]] = 1
        elif load_bus[0] in load_order:
            load_order[load_bus[0]] += 1
    return load_order
load_order_list = load_order_f()

def load_values():
    load_ph_arr = np.zeros((nnode, max(load_order_list.values()), 3))
    load_kw_arr_ph = np.zeros((3, nnode))
    load_kvar_arr_ph = np.zeros((3, nnode))
    if t == -1:
        var = 1
    else:
        var = (1 + 0.1*np.sin(2*np.pi*0.01*t))
    for load in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[load])
        load_data = dss.CktElement.BusNames()[0].split('.')
        idxbs = dss.Circuit.AllBusNames().index(load_data[0])
        for ph in range(1, len(load_data)):
            load_kw_arr_ph[int(load_data[ph]) - 1, idxbs] += dss.Loads.kW() * 1e3 / Sbase / (len(load_data) - 1)
            load_kvar_arr_ph[int(load_data[ph]) - 1, idxbs] += dss.Loads.kvar() * 1e3 / Sbase / (len(load_data) - 1) 
    return load_kw_arr_ph, load_kvar_arr_ph
load_kw_arr_ph, load_kvar_arr_ph = load_values()
print(load_kw_arr_ph)
print(load_kvar_arr_ph)

In [ ]:

            idxbs = dss.Circuit.AllBusNames().index(cap_data[0])
            for ph in range(1, len(cap_data)):
                caparr[int(cap_data[ph]) - 1, idxbs] += dss.Capacitors.kvar() * 1e3 / Sbase / (len(cap_data) - 1)
        return caparr

In [ ]:
[[0.         0.         0.10384609 0.10384609 0.10384609 0.15576913
  0.        ]
 [0.         0.         0.10384609 0.10384609 0.         0.10384609
  0.05192304]
 [0.         0.         0.10384609 0.10384609 0.10384609 0.
  0.        ]]
[[0.         0.         0.02596152 0.02596152 0.02596152 0.05192304
  0.        ]
 [0.         0.         0.02596152 0.02596152 0.         0.02596152
  0.02596152]
 [0.         0.         0.02596152 0.02596152 0.02596152 0.
  0.        ]]